<a href="https://colab.research.google.com/github/Saakshitha/Secpen_model/blob/main/Hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install transformers datasets torch scikit-learn gradio


In [2]:

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

filename = list(uploaded.keys())[0]

Saving df_balanced.csv to df_balanced.csv
User uploaded file "df_balanced.csv" with length 70331 bytes


In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset

df = pd.read_csv(filename)

le = LabelEncoder()
df["label"] = le.fit_transform(df["Sentiment"])
emotion_labels = list(le.classes_)

dataset = Dataset.from_pandas(df[["Text", "label"]])
dataset = dataset.train_test_split(test_size=0.1, seed=42)


In [4]:
from transformers import AutoTokenizer

deberta_tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
bertweet_tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")

def tokenize_deberta(batch):
    return deberta_tokenizer(batch["Text"], truncation=True, padding="max_length", max_length=128)

def tokenize_bertweet(batch):
    return bertweet_tokenizer(batch["Text"], truncation=True, padding="max_length", max_length=128)

deberta_ds = dataset.map(tokenize_deberta, batched=True)
bertweet_ds = dataset.map(tokenize_bertweet, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


...

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


Map:   0%|          | 0/658 [00:00<?, ? examples/s]

Map:   0%|          | 0/74 [00:00<?, ? examples/s]

Map:   0%|          | 0/658 [00:00<?, ? examples/s]

Map:   0%|          | 0/74 [00:00<?, ? examples/s]

In [5]:
import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = len(emotion_labels)

# DeBERTa Model
deberta_model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-base", num_labels=num_labels)

# BERTweet Model
bertweet_model = AutoModelForSequenceClassification.from_pretrained(
    "vinai/bertweet-base", num_labels=num_labels)

training_args = TrainingArguments(
    output_dir="deberta-emotion",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    max_steps=100,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# Metrics
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }



pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# DeBERTa Trainer
deberta_trainer = Trainer(
    model=deberta_model,
    args=training_args,
    train_dataset=deberta_ds["train"],
    eval_dataset=deberta_ds["test"],
    tokenizer=deberta_tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-6-5a31c1ddba49>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  deberta_trainer = Trainer(


In [7]:
# BERTweet Trainer
bertweet_trainer = Trainer(
    model=bertweet_model,
    args=training_args,
    train_dataset=bertweet_ds["train"],
    eval_dataset=bertweet_ds["test"],
    tokenizer=bertweet_tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-7-f3b43f44a786>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  bertweet_trainer = Trainer(


In [ ]:
deberta_trainer.train()
#3eb1181871c09ea302f55e200422da7d392c1dde

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: saakshitha2004 (saakshitha2004-secpen) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


In [ ]:
bertweet_trainer.train()

In [ ]:
import torch

class HybridEmotionClassifier:
    def __init__(self, deberta_model, bertweet_model, deberta_tokenizer, bertweet_tokenizer, labels):
        self.deberta_model = deberta_model.eval()
        self.bertweet_model = bertweet_model.eval()
        self.deberta_tokenizer = deberta_tokenizer
        self.bertweet_tokenizer = bertweet_tokenizer
        self.labels = labels
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.deberta_model.to(self.device)
        self.bertweet_model.to(self.device)

    def predict(self, text):
        # DeBERTa
        d_inputs = self.deberta_tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(self.device)
        with torch.no_grad():
            d_logits = self.deberta_model(**d_inputs).logits
        # BERTweet
        b_inputs = self.bertweet_tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(self.device)
        with torch.no_grad():
            b_logits = self.bertweet_model(**b_inputs).logits
        # Average logits
        avg_logits = (d_logits + b_logits) / 2
        probs = torch.softmax(avg_logits, dim=1).cpu().numpy()[0]
        pred_idx = probs.argmax()
        return {
            "emotion": self.labels[pred_idx],
            "confidence": float(probs[pred_idx]),
            "probabilities": dict(zip(self.labels, map(float, probs)))
        }

# Instantiate hybrid classifier
hybrid_classifier = HybridEmotionClassifier(
    deberta_model, bertweet_model, deberta_tokenizer, bertweet_tokenizer, emotion_labels
)


In [ ]:
import gradio as gr

def detect_emotion(text):
    result = hybrid_classifier.predict(text)
    return f"Emotion: {result['emotion']}\nConfidence: {result['confidence']:.2f}\n\nFull probabilities:\n" + \
           "\n".join([f"{k}: {v:.2f}" for k, v in result["probabilities"].items()])

gr.Interface(
    fn=detect_emotion,
    inputs=gr.Textbox(lines=2, placeholder="Enter text here..."),
    outputs="text",
    title="Hybrid Emotion Detector",
    description="Fine-tuned DeBERTa-v3 and BERTweet Ensemble"
).launch(share=True)
